In [3]:
# Bootstrap minimal deps for parsing (no API/env)
import sys, subprocess

# Ensure lxml is available for pandas.read_html / BeautifulSoup
try:
    import lxml  # type: ignore
    print('lxml already available')
except Exception:
    print('Installing lxml…')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'lxml'])
    import lxml  # type: ignore
    print('lxml installed')


Installing lxml…
lxml installed


In [7]:
# Try requests + BeautifulSoup first (no JS)
import os
from pathlib import Path
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Target URL from user (encodes date=22.11.2025 and NO1 area, aFRR A96)
NUCS_UI_URL = (
    "https://www.nucs.net/balancing/r2/pricesAndVolumesOfProcuredBalancingReserve/show"
    "?name=&defaultValue=false&viewType=TABLE&areaType=MBA&atch=false"
    "&dateTime.dateTime=22.11.2025+00:00|CET|DAYTIMERANGE"
    "&dateTime.endDateTime=22.11.2025+00:00|CET|DAYTIMERANGE"
    "&areaSelectType=USER_SELECTED"
    "&marketArea.values=CTY|10YNO-0--------C!MBA|10YNO-1--------2"
    "&balancingDirection.values=A01"
    "&dataItems.values=PRICE"
    "&reserveType.values=A96"
    "&balancingTypes=SECONDARY"
    "&reserveSource.values=ALL"
    "&aFRRmFRRType.values=A51"
)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                  " AppleWebKit/537.36 (KHTML, like Gecko)"
                  " Chrome/120.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

FOUND_TABLES = False
tables = []
resp = None
try:
    resp = requests.get(NUCS_UI_URL, headers=headers, timeout=60)
    print("GET status:", resp.status_code, "| length:", len(resp.text))
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "lxml")
    # Try pandas to find HTML tables
    try:
        tables = pd.read_html(resp.text)
    except ValueError:
        tables = []
    if tables:
        FOUND_TABLES = True
        # Prefer the largest table by row count
        df = max(tables, key=lambda d: (len(d.index), len(d.columns)))
        print("Parsed table shape:", df.shape)
        # Persist CSV
        out_dir = Path('..') / 'reports' / 'dataframes'
        out_dir.mkdir(parents=True, exist_ok=True)
        out_csv = out_dir / 'nucs_ui_prices_NO1_2025-11-22_a96.csv'
        df.to_csv(out_csv, index=False)
        print('Saved:', out_csv)
        display(df.head(10))
    else:
        print("No tables found in static HTML. The page may require JS rendering.")
except Exception as e:
    print("requests/bs4 path failed:", repr(e))
    FOUND_TABLES = False


GET status: 200 | length: 80852
requests/bs4 path failed: FeatureNotFound("Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?")


In [8]:
# Selenium fallback (headless): render then parse table
import sys
import time
import pandas as pd

if 'FOUND_TABLES' not in globals():
    FOUND_TABLES = False

if not FOUND_TABLES:
    print('Attempting Selenium fallback (headless)…')
    try:
        try:
            import selenium  # type: ignore
        except ImportError:
            print('Installing selenium + webdriver-manager…')
            import subprocess
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'selenium', 'webdriver-manager'])  # noqa: E501
        from selenium import webdriver  # type: ignore
        from selenium.webdriver.chrome.service import Service as ChromeService  # type: ignore
        from selenium.webdriver.common.by import By  # type: ignore
        from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
        from selenium.webdriver.support import expected_conditions as EC  # type: ignore
        from webdriver_manager.chrome import ChromeDriverManager  # type: ignore
        options = webdriver.ChromeOptions()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        try:
            driver.get(NUCS_UI_URL)
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
            time.sleep(1)
            html = driver.page_source
        finally:
            driver.quit()
        try:
            tables2 = pd.read_html(html)
        except ValueError:
            tables2 = []
        if tables2:
            df2 = max(tables2, key=lambda d: (len(d.index), len(d.columns)))
            print('Rendered table shape:', df2.shape)
            from pathlib import Path
            out_dir = Path('..') / 'reports' / 'dataframes'
            out_dir.mkdir(parents=True, exist_ok=True)
            out_csv = out_dir / 'nucs_ui_prices_NO1_2025-11-22_a96_selenium.csv'
            df2.to_csv(out_csv, index=False)
            print('Saved:', out_csv)
            display(df2.head(10))
        else:
            print('Selenium rendered HTML, but no tables were detected with pandas.read_html().')
    except Exception as e:
        print('Selenium fallback failed:', repr(e))
else:
    print('Static parse already succeeded; skipping Selenium.')


Attempting Selenium fallback (headless)…
Rendered table shape: (24, 3)
Saved: ..\reports\dataframes\nucs_ui_prices_NO1_2025-11-22_a96_selenium.csv


C:\Users\haako\AppData\Local\Temp\ipykernel_19440\2524715902.py:39: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables2 = pd.read_html(html)


,Day,Hour,MBA|NO1
,Day,Hour,UP
,Day,Hour,Price [EUR/MW]
0,22.11.2025,00:00,9.87
1,NaN,01:00,7.70
2,NaN,02:00,6.50
3,NaN,03:00,6.00
4,NaN,04:00,5.70
5,NaN,05:00,4.50
6,NaN,06:00,5.80
7,NaN,07:00,6.30


In [3]:
# Scrape full year for NO1 (aFRR A96): configurable date range (PRICE + VOLUME, reserveSource=ALL)
import sys, time
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd

# Ensure selenium and driver utilities are present (reusing earlier installs if available)
try:
    from selenium import webdriver  # type: ignore
    from selenium.webdriver.chrome.service import Service as ChromeService  # type: ignore
    from selenium.webdriver.common.by import By  # type: ignore
    from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
    from selenium.webdriver.support import expected_conditions as EC  # type: ignore
    from webdriver_manager.chrome import ChromeDriverManager  # type: ignore
except Exception as e:
    print('Installing selenium + webdriver-manager…')
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'selenium', 'webdriver-manager'])
    from selenium import webdriver  # type: ignore
    from selenium.webdriver.chrome.service import Service as ChromeService  # type: ignore
    from selenium.webdriver.common.by import By  # type: ignore
    from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
    from selenium.webdriver.support import expected_conditions as EC  # type: ignore
    from webdriver_manager.chrome import ChromeDriverManager  # type: ignore

def build_url(day_dt: datetime) -> str:
    d = day_dt.strftime('%d.%m.%Y')
    base = (
        'https://www.nucs.net/balancing/r2/pricesAndVolumesOfProcuredBalancingReserve/show'
        '?name=&defaultValue=false&viewType=TABLE&areaType=MBA&atch=false'
        f'&dateTime.dateTime={d}+00:00|CET|DAYTIMERANGE'
        f'&dateTime.endDateTime={d}+00:00|CET|DAYTIMERANGE'
        '&areaSelectType=USER_SELECTED'
        '&marketArea.values=CTY|10YNO-0--------C!MBA|10YNO-1--------2'
        '&balancingDirection.values=A01'
        # Request both PRICE and VOLUME
        '&dataItems.values=PRICE&dataItems.values=VOLUME'
        '&reserveType.values=A96'
        '&balancingTypes=SECONDARY'
        '&reserveSource.values=ALL'
        '&aFRRmFRRType.values=A51'
    )
    return base

def parse_table_from_html(html: str, day_dt: datetime) -> pd.DataFrame:
    dfs = []
    try:
        dfs = pd.read_html(html)
    except Exception:
        return pd.DataFrame()
    if not dfs:
        return pd.DataFrame()
    df = max(dfs, key=lambda d: (len(d.index), len(d.columns)))
    # Flatten possible MultiIndex columns by taking the last level name
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [str(c[-1]) for c in df.columns.values]
    else:
        df.columns = [str(c) for c in df.columns]
    # Heuristics to find key columns
    col_day = next((c for c in df.columns if c.strip().lower() in ('day','date')), None)
    col_hour = next((c for c in df.columns if c.strip().lower().startswith('hour')), None)
    price_cols = [c for c in df.columns if 'price' in c.lower()]
    vol_cols   = [c for c in df.columns if 'volume' in c.lower()]
    col_price = price_cols[-1] if price_cols else None
    col_volume = vol_cols[-1] if vol_cols else None
    if not col_hour or not (col_price or col_volume):
        return pd.DataFrame()
    # Many tables only show the day on the first row; ffill it and fallback to explicit day
    if col_day and col_day in df.columns:
        df[col_day] = df[col_day].ffill().fillna(day_dt.strftime('%d.%m.%Y'))
        day_str = df[col_day].iloc[0]
    else:
        day_str = day_dt.strftime('%d.%m.%Y')
        df[col_day or 'Day'] = day_str
    # Build base frame
    cols = [c for c in [col_day or 'Day', col_hour, col_price, col_volume] if c]
    use = df[cols].copy()
    # Rename
    rename_map = {}
    if (col_day or 'Day') in use.columns:
        rename_map[col_day or 'Day'] = 'day'
    if col_hour in use.columns:
        rename_map[col_hour] = 'hour'
    if col_price and col_price in use.columns:
        rename_map[col_price] = 'price_raw'
    if col_volume and col_volume in use.columns:
        rename_map[col_volume] = 'volume_raw'
    use = use.rename(columns=rename_map)
    # Normalize numeric
    for num_col, out_col in [('price_raw','price'), ('volume_raw','volume')]:
        if num_col in use.columns:
            tmp = (use[num_col].astype(str)
                           .str.replace('\u00a0','', regex=False)
                           .str.replace(' ','', regex=False)
                           .str.replace(',','.', regex=False))
            use[out_col] = pd.to_numeric(tmp, errors='coerce')
    # Build timestamp (local CET label per UI)
    use['date'] = day_dt.strftime('%Y-%m-%d')
    use['ts_local'] = pd.to_datetime(use['date'] + ' ' + use['hour'], format='%Y-%m-%d %H:%M', errors='coerce')
    # Metadata
    use['area'] = 'NO1'
    use['reserveType'] = 'A96'
    use['direction'] = 'A01'  # UP
    # Final columns (include volume if present)
    final_cols = ['ts_local','date','hour','price','volume','area','reserveType','direction']
    for c in final_cols:
        if c not in use.columns:
            use[c] = pd.NA
    return use[final_cols]

# Quick test: one day given by you (01.11.2024)
options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1600,1200')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
try:
    test_day = datetime(2024, 11, 1)
    driver.get(build_url(test_day))
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
    html = driver.page_source
    df_test = parse_table_from_html(html, test_day)
    print('Test rows:', len(df_test))
    out_dir = Path('..') / 'reports' / 'dataframes'
    out_dir.mkdir(parents=True, exist_ok=True)
    out_csv = out_dir / 'nucs_ui_prices_volumes_NO1_2024-11-01_a96_selenium.csv'
    df_test.to_csv(out_csv, index=False)
    print('Saved:', out_csv)
    display(df_test.head(12))
finally:
    try:
        driver.quit()
    except Exception:
        pass

Test rows: 24
Saved: ..\reports\dataframes\nucs_ui_prices_volumes_NO1_2024-11-01_a96_selenium.csv


C:\Users\haako\AppData\Local\Temp\ipykernel_22148\2532347867.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


,ts_local,date,hour,price,volume,area,reserveType,direction
0,2024-11-01 00:00:00,2024-11-01,00:00,16.00,15,NO1,A96,A01
1,2024-11-01 01:00:00,2024-11-01,01:00,12.00,0,NO1,A96,A01
2,2024-11-01 02:00:00,2024-11-01,02:00,12.00,0,NO1,A96,A01
3,2024-11-01 03:00:00,2024-11-01,03:00,12.00,0,NO1,A96,A01
4,2024-11-01 04:00:00,2024-11-01,04:00,13.00,0,NO1,A96,A01
5,2024-11-01 05:00:00,2024-11-01,05:00,29.12,15,NO1,A96,A01
6,2024-11-01 06:00:00,2024-11-01,06:00,32.50,15,NO1,A96,A01
7,2024-11-01 07:00:00,2024-11-01,07:00,31.50,15,NO1,A96,A01
8,2024-11-01 08:00:00,2024-11-01,08:00,31.50,15,NO1,A96,A01
9,2024-11-01 09:00:00,2024-11-01,09:00,20.20,15,NO1,A96,A01


In [4]:
# Full 2025 run (incremental write) — NO1 aFRR (A96), UP (A01), PRICE+VOLUME, reserveSource=ALL
import os, sys, time, random
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd

# Reuse helpers defined earlier: build_url(day_dt), parse_table_from_html(html, day_dt)
try:
    build_url, parse_table_from_html
except NameError:
    raise RuntimeError('Please run the previous cell that defines build_url and parse_table_from_html first.')

# Ensure selenium imports present
try:
    from selenium import webdriver  # type: ignore
    from selenium.webdriver.chrome.service import Service as ChromeService  # type: ignore
    from selenium.webdriver.common.by import By  # type: ignore
    from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
    from selenium.webdriver.support import expected_conditions as EC  # type: ignore
    from webdriver_manager.chrome import ChromeDriverManager  # type: ignore
except Exception as e:
    print('Installing selenium + webdriver-manager…')
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'selenium', 'webdriver-manager'])
    from selenium import webdriver  # type: ignore
    from selenium.webdriver.chrome.service import Service as ChromeService  # type: ignore
    from selenium.webdriver.common.by import By  # type: ignore
    from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
    from selenium.webdriver.support import expected_conditions as EC  # type: ignore
    from webdriver_manager.chrome import ChromeDriverManager  # type: ignore

DATE_START = datetime(2025, 1, 1)
DATE_END   = datetime(2025, 12, 31)  # inclusive
PAUSE_SEC  = 10.0  # base politeness pause (slower per request)
JITTER_SEC = 2.0   # small random extra pause
MAX_RETRY  = 2     # per day
MAX_CONSEC_FAILS = 5  # abort after N consecutive server/UI errors
BACKOFF_BASE = 2.0  # exponential backoff multiplier per consecutive failure

# Build date list
dates = []
cur = DATE_START
while cur <= DATE_END:
    dates.append(cur)
    cur += timedelta(days=1)
print(f'Running full year: {len(dates)} days')

out_dir = Path('..') / 'reports' / 'dataframes'
out_dir.mkdir(parents=True, exist_ok=True)
agg_path = out_dir / 'nucs_ui_prices_volumes_NO1_2025_a96_selenium.csv'
print(('Appending to existing file:' if agg_path.exists() else 'Creating new file:'), agg_path)

# Resume-skip: if a day already written (by ts_local date), skip
written_dates = set()
if agg_path.exists():
    try:
        df_existing = pd.read_csv(agg_path, usecols=['date'])
        written_dates = set(df_existing['date'].astype(str).unique())
        print('Resume-skip enabled. Already have', len(written_dates), 'dates.')
    except Exception as e:
        print('Could not read existing CSV for resume; will append without skip:', repr(e))

options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1600,1200')
options.add_argument('--disable-dev-shm-usage')
# Mild user-agent variation
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36')

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

written_rows = 0
consec_fail = 0
try:
    for i, day_dt in enumerate(dates, 1):
        day_str = day_dt.strftime('%Y-%m-%d')
        if day_str in written_dates:
            print(f"[{i:03d}/{len(dates)}] {day_str} already in CSV; skipping.")
            continue
        url = build_url(day_dt)
        ok = False
        df_day = pd.DataFrame()
        for attempt in range(1, MAX_RETRY+1):
            try:
                driver.get(url)
                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
                html = driver.page_source
                # Detect server-side UI error page
                if ('AJAX_ERROR' in html) or ('Unable to send request to server' in html) or ('Error occurred' in html):
                    raise RuntimeError('UI error marker detected (AJAX_ERROR/Unable to send request).')
                df_day = parse_table_from_html(html, day_dt)
                if not df_day.empty:
                    ok = True
                    break
                else:
                    print(f"[{i:03d}/{len(dates)}] {day_str} no rows parsed; retry {attempt}/{MAX_RETRY}")
                    time.sleep(1.2)
            except Exception as e:
                print(f"[{i:03d}/{len(dates)}] {day_str} error: {repr(e)}; retry {attempt}/{MAX_RETRY}")
                time.sleep(1.2)
        if ok and not df_day.empty:
            # Append incrementally to CSV
            header = not agg_path.exists()
            df_day.to_csv(agg_path, mode='a', header=header, index=False)
            written_rows += len(df_day)
            consec_fail = 0
            written_dates.add(day_str)
            print(f"[{i:03d}/{len(dates)}] {day_str} wrote {len(df_day)} rows (total {written_rows})")
        else:
            consec_fail += 1
            print(f"[{i:03d}/{len(dates)}] {day_str} skipped (no data). Consecutive fails: {consec_fail}")
            if consec_fail >= MAX_CONSEC_FAILS:
                print('Too many consecutive failures; stopping to avoid stressing the site. Try later.')
                break
        # Politeness sleep with jitter and exponential backoff on consecutive failures
        sleep_s = PAUSE_SEC + random.uniform(0, JITTER_SEC) + (0 if consec_fail == 0 else min(60, BACKOFF_BASE ** consec_fail))
        time.sleep(sleep_s)
    print('Finished. Aggregate CSV at:', agg_path, '| total rows appended this run:', written_rows)
except KeyboardInterrupt:
    print('Interrupted by user. Partial data saved at:', agg_path, '| rows so far this run:', written_rows)
finally:
    try:
        driver.quit()
    except Exception:
        pass

Running full year: 365 days
Creating new file: ..\reports\dataframes\nucs_ui_prices_volumes_NO1_2025_a96_selenium.csv


C:\Users\haako\AppData\Local\Temp\ipykernel_22148\2532347867.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


[001/365] 2025-01-01 wrote 24 rows (total 24)


C:\Users\haako\AppData\Local\Temp\ipykernel_22148\2532347867.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


[002/365] 2025-01-02 wrote 24 rows (total 48)


...

[363/365] 2025-12-29 wrote 24 rows (total 8712)


C:\Users\haako\AppData\Local\Temp\ipykernel_22148\2532347867.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


[364/365] 2025-12-30 wrote 24 rows (total 8736)


C:\Users\haako\AppData\Local\Temp\ipykernel_22148\2532347867.py:48: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)


[365/365] 2025-12-31 wrote 24 rows (total 8760)
Finished. Aggregate CSV at: ..\reports\dataframes\nucs_ui_prices_volumes_NO1_2025_a96_selenium.csv | total rows appended this run: 8760
Finished. Aggregate CSV at: ..\reports\dataframes\nucs_ui_prices_volumes_NO1_2025_a96_selenium.csv | total rows appended this run: 8760


In [1]:
# Health check: detect server/UI error before scraping
import requests
from datetime import datetime
from pathlib import Path

def health_check(day_dt: datetime) -> None:
    d = day_dt.strftime('%d.%m.%Y')
    url = ("https://www.nucs.net/balancing/r2/pricesAndVolumesOfProcuredBalancingReserve/show"
           "?name=&defaultValue=false&viewType=TABLE&areaType=MBA&atch=false"
           f"&dateTime.dateTime={d}+00:00|CET|DAYTIMERANGE"
           f"&dateTime.endDateTime={d}+00:00|CET|DAYTIMERANGE"
           "&areaSelectType=USER_SELECTED"
           "&marketArea.values=CTY|10YNO-0--------C!MBA|10YNO-1--------2"
           "&balancingDirection.values=A01"
           "&dataItems.values=PRICE"
           "&reserveType.values=A96"
           "&balancingTypes=SECONDARY"
           "&reserveSource.values=ALL"
           "&aFRRmFRRType.values=A51")
    try:
        r = requests.get(url, timeout=30)
        print('Status:', r.status_code, '| bytes:', len(r.content))
        text = r.text[:2000]
        if 'AJAX_ERROR' in text or 'Unable to send request to server' in text or 'Error occurred' in text:
            print('UI indicates server/ajax error; recommend pausing and retrying later.')
        else:
            print('No obvious UI error markers in first 2KB of HTML.')
    except Exception as e:
        print('Health check request failed:', repr(e))

# Example: check the date that currently fails for you
health_check(datetime(2025, 11, 22))

Health check request failed: ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='www.nucs.net', port=443): Max retries exceeded with url: /balancing/r2/pricesAndVolumesOfProcuredBalancingReserve/show?name=&defaultValue=false&viewType=TABLE&areaType=MBA&atch=false&dateTime.dateTime=22.11.2025+00:00%7CCET%7CDAYTIMERANGE&dateTime.endDateTime=22.11.2025+00:00%7CCET%7CDAYTIMERANGE&areaSelectType=USER_SELECTED&marketArea.values=CTY%7C10YNO-0--------C!MBA%7C10YNO-1--------2&balancingDirection.values=A01&dataItems.values=PRICE&reserveType.values=A96&balancingTypes=SECONDARY&reserveSource.values=ALL&aFRRmFRRType.values=A51 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002C6FBC49F40>, 'Connection to www.nucs.net timed out. (connect timeout=30)'))"))
